# OpenEvolve Standalone Tutorial

This notebook demonstrates how to use **GEAK-OpenEvolve** for GPU kernel optimization using LLM-guided evolution.

## Prerequisites
 **Environment Variables**: Set `OPENAI_API_KEY`

## What You'll Learn

- How to set up GEAK-OpenEvolve
- How to prepare an initial kernel program
- How to configure evolution parameters
- How to run the evolution pipeline
- How to analyze results

In [1]:
# Step 1: Environment Setup
import os
import sys
from pathlib import Path

# Get geak-openevolve root
OPENEVOLVE_ROOT = Path.cwd().parent
print(f"OpenEvolve Root: {OPENEVOLVE_ROOT}")

# Add to Python path
if str(OPENEVOLVE_ROOT) not in sys.path:
    sys.path.insert(0, str(OPENEVOLVE_ROOT))

print(f"\n✅ OpenEvolve root: {OPENEVOLVE_ROOT}")
print(f"✅ Python path updated")


OpenEvolve Root: /home/sapmajum/neurips/geak-openevolve

✅ OpenEvolve root: /home/sapmajum/neurips/geak-openevolve
✅ Python path updated


In [2]:
# Install all required packages
!pip install -q ipykernel
!python3 -m pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/rocm6.2.4/
# PyTorch for ROCm (gfx94X)
!python -m pip install https://rocm.nightlies.amd.com/v2/gfx94X-dcgpu/torch/torch-2.7.0a0+rocm7.0.0rc20250711-cp312-cp312-linux_x86_64.whl

# Triton 3.3.0
!python -m pip install -U triton==3.3.0

# Other dependencies
!pip install -q pyyaml openai pytest pytest-timeout
!pip install tenacity loguru parse_llm_code rank_bm25

print('✅ All dependencies installed!')

Looking in indexes: https://download.pytorch.org/whl/nightly/rocm6.2.4/
ERROR: torch-2.7.0a0+rocm7.0.0rc20250711-cp312-cp312-linux_x86_64.whl is not a supported wheel on this platform.
✅ All dependencies installed!


In [3]:
# Step 1.5: Clone and Install GEAK-eval (if not already done)
import os
from pathlib import Path
import subprocess

OPENEVOLVE_ROOT = Path.cwd().parent
GEAK_EVAL_DIR = OPENEVOLVE_ROOT / "GEAK-eval-OE"

if not GEAK_EVAL_DIR.exists():
    print("📥 Cloning GEAK-eval...")
    os.chdir(OPENEVOLVE_ROOT)
    
    # Clone and checkout
    subprocess.run(["git", "clone", "git@github.com:AMD-AGI/GEAK-eval.git", "GEAK-eval-OE"], check=True)
    os.chdir("GEAK-eval-OE")
    subprocess.run(["git", "checkout", "geak-oe"], check=True)
    
    print("✅ GEAK-eval cloned")
    
    # Install
    print("📦 Installing GEAK-eval...")
    subprocess.run(["pip", "install", "-e", ".", "--no-deps"], check=True)
    print("✅ GEAK-eval installed")
else:
    print(f"✅ GEAK-eval already exists at: {GEAK_EVAL_DIR}")
    
    # Check if installed
    try:
        result = subprocess.run(["which", "geak-eval"], capture_output=True, text=True)
        if result.returncode == 0:
            print(f"✅ geak-eval command available: {result.stdout.strip()}")
        else:
            print("⚠️  geak-eval command not found, installing...")
            os.chdir(GEAK_EVAL_DIR)
            subprocess.run(["pip", "install", "-e", ".", "--no-deps"], check=True)
            print("✅ GEAK-eval installed")
    except Exception as e:
        print(f"⚠️  Could not check geak-eval: {e}")


✅ GEAK-eval already exists at: /home/sapmajum/neurips/geak-openevolve/GEAK-eval-OE
✅ geak-eval command available: /home/sapmajum/.local/bin/geak-eval


In [4]:
# Step 2: Set Environment Variables
import os
from pathlib import Path

# Set API key
os.environ['OPENAI_API_KEY'] = "<your-api-here>"

# Set ROCM_GOLDEN_DATA_PATH
OPENEVOLVE_ROOT = Path.cwd().parent
GOLDEN_DATA_PATH = OPENEVOLVE_ROOT / "GEAK-eval-OE/geak_eval/data/ROCm/data/performance/golden_results"
os.environ['ROCM_GOLDEN_DATA_PATH'] = str(GOLDEN_DATA_PATH)

print(f"✅ OPENAI_API_KEY set")
print(f"✅ ROCM_GOLDEN_DATA_PATH = {GOLDEN_DATA_PATH}")
print(f"   Path exists: {GOLDEN_DATA_PATH.exists()}")


✅ OPENAI_API_KEY set
✅ ROCM_GOLDEN_DATA_PATH = /home/sapmajum/neurips/geak-openevolve/GEAK-eval-OE/geak_eval/data/ROCm/data/performance/golden_results
   Path exists: True


In [5]:
# Step 3: Verify OpenEvolve Installation
import sys
import torch

print(f"Python: {sys.version.split()[0]}")
print(f"PyTorch: {torch.__version__}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'N/A'}")

try:
    import triton
    print(f"Triton: {triton.__version__}")
except:
    print("❌ Triton not found")

try:
    import openevolve
    print(f"OpenEvolve: {openevolve.__version__ if hasattr(openevolve, '__version__') else 'installed'}")
except:
    print("❌ OpenEvolve not found - install with: pip install -e .")

print("\n✅ Environment ready!")


Python: 3.13.9
PyTorch: 2.7.0.dev20250310+rocm6.2.4
GPU: AMD Instinct MI325X
Triton: 3.2.0
OpenEvolve: 0.1.0

✅ Environment ready!


## Kernel Preparation

OpenEvolve requires:
1. **Initial Kernel**: The starting kernel code to optimize
2. **Evaluator**: A function that evaluates kernel performance
3. **Configuration**: Evolution parameters (iterations, population size, etc.)

We'll use a validated ROCm Triton kernel as our example.


In [6]:
# Step 4: Select Example Kernel
from pathlib import Path

OPENEVOLVE_ROOT = Path.cwd().parent
TUTORIAL_DIR = OPENEVOLVE_ROOT / "tutorial"

# Use kernel from GEAK-eval-OE (cloned GEAK-eval repository)
INITIAL_KERNEL = OPENEVOLVE_ROOT / "GEAK-eval-OE/geak_eval/data/ROCm/data/ROCm_v1/test_add_kernel.py"

if INITIAL_KERNEL.exists():
    print(f"✅ Selected kernel: {INITIAL_KERNEL.name}")
    print(f"   Path: {INITIAL_KERNEL.relative_to(OPENEVOLVE_ROOT)}")
    
    # Quick peek at the kernel
    with open(INITIAL_KERNEL, 'r') as f:
        lines = f.readlines()
    
    # Find the kernel function
    in_kernel = False
    kernel_lines = []
    for line in lines:
        if '@triton.jit' in line:
            in_kernel = True
        if in_kernel:
            kernel_lines.append(line.rstrip())
            if line.strip().startswith('tl.store') and 'output' in line:
                break
    
    print(f"\n📝 Kernel Preview:")
    for line in kernel_lines[:15]:
        print(f"   {line}")
    if len(kernel_lines) > 15:
        print(f"   ... ({len(kernel_lines)-15} more lines)")
else:
    print(f"❌ Kernel not found at: {INITIAL_KERNEL}")
    INITIAL_KERNEL = None


✅ Selected kernel: test_add_kernel.py
   Path: GEAK-eval-OE/geak_eval/data/ROCm/data/ROCm_v1/test_add_kernel.py

📝 Kernel Preview:
   @triton.jit
   def add_kernel(
       x_ptr,
       y_ptr,
       output_ptr,
       n_elements,
       BLOCK_SIZE: tl.constexpr,
   ):
       pid = tl.program_id(axis=0)  # We use a 1D launch grid so axis is 0.
       block_start = pid * BLOCK_SIZE
       offsets = block_start + tl.arange(0, BLOCK_SIZE)
       mask = offsets < n_elements
   
       x_block_ptr = tl.make_block_ptr(base=x_ptr, shape=(n_elements, ), strides=(1, ), offsets=(pid * BLOCK_SIZE, ),
                                       block_shape=(BLOCK_SIZE, ), order=(0, ))
   ... (5 more lines)


In [7]:
# Step 5: Setup Evaluator
from pathlib import Path

OPENEVOLVE_ROOT = Path.cwd().parent

# Use the ROCm evaluator from examples
EVALUATOR_PATH = OPENEVOLVE_ROOT / "examples/tb/rocm_evaluator.py"

if EVALUATOR_PATH.exists():
    print(f"✅ Using evaluator: {EVALUATOR_PATH.name}")
    print(f"   Path: {EVALUATOR_PATH.relative_to(OPENEVOLVE_ROOT)}")
else:
    print(f"❌ Evaluator not found at: {EVALUATOR_PATH}")
    EVALUATOR_PATH = None


✅ Using evaluator: rocm_evaluator.py
   Path: examples/tb/rocm_evaluator.py


In [8]:
# Step 6: Configure Evolution Parameters
import yaml
from pathlib import Path

OPENEVOLVE_ROOT = Path.cwd().parent
TUTORIAL_DIR = OPENEVOLVE_ROOT / "tutorial"

# Configuration parameters - EASILY ADJUSTABLE
MAX_ITERATIONS = 10
POPULATION_SIZE = 50
NUM_ISLANDS = 4
LOG_LEVEL = "WARNING"

# Try multiple config templates
CONFIG_TEMPLATES = [
    OPENEVOLVE_ROOT / "configs/default_config.yaml",
    OPENEVOLVE_ROOT / "examples/tb/configs/demo_config.yaml",
]

CONFIG_FILE = TUTORIAL_DIR / "tutorial_config.yaml"

# Find first available template
template_found = None
for template in CONFIG_TEMPLATES:
    if template.exists():
        template_found = template
        print(f"✅ Found config template: {template.relative_to(OPENEVOLVE_ROOT)}")
        break

if template_found:
    with open(template_found, 'r') as f:
        config = yaml.safe_load(f)
    
    config['max_iterations'] = MAX_ITERATIONS
    config['log_level'] = LOG_LEVEL
    
    if 'database' not in config:
        config['database'] = {}
    config['database']['population_size'] = POPULATION_SIZE
    config['database']['num_islands'] = NUM_ISLANDS
    config['database']['log_prompts'] = True
    
    # CRITICAL: Fix db_path (can't be None)
    if config['database'].get('db_path') is None:
        config['database']['db_path'] = 'program_database'
    
    if 'llm' not in config:
        config['llm'] = {}
    
    # CRITICAL: Set sampling configuration
    config['llm']['sampling'] = {'fn': 'random'}
    
    config['llm']['models'] = [{'name': 'claude-sonnet-4-5', 'weight': 1.0}]
    config['llm']['evaluator_models'] = [{'name': 'claude-sonnet-4-5', 'weight': 1.0}]
    config['llm']['api_base'] = 'https://llm-api.amd.com/AnthropicVertex/deployments/claude-sonnet-4-5'
    config['llm']['api_key'] = None
    
    if 'evaluator' not in config:
        config['evaluator'] = {}
    config['evaluator']['cascade_evaluation'] = False
    config['evaluator']['verbose'] = False
    
    # Set prompt template directory for advanced prompts
    if 'prompt' not in config:
        config['prompt'] = {}
    config['prompt']['template_dir'] = './prompts_tutorial'
    # Remove inline prompts when using template_dir (they would override template files)
    config['prompt'].pop('system_message', None)
    config['prompt'].pop('evaluator_system_message', None)
    
    # Set LLM parameters for code generation
    config['llm']['max_tokens'] = 10000
    config['llm']['timeout'] = 200
    
    config['diff_based_evolution'] = True
    config['max_code_length'] = 50000
    config['evaluator']['use_llm_feedback'] = True
    config['evaluator']['parallel_evaluations'] = 1
    
    # CRITICAL: Create evals directory for evaluator temp files
    evals_dir = TUTORIAL_DIR / "evals"
    evals_dir.mkdir(exist_ok=True)
    
    with open(CONFIG_FILE, 'w') as f:
        yaml.dump(config, f, default_flow_style=False, sort_keys=False)
    
    print(f"✅ Configuration saved to: {CONFIG_FILE.name}")
    print(f"\n📝 Evolution Parameters:")
    print(f"  Max Iterations:  {MAX_ITERATIONS}")
    print(f"  Population Size: {POPULATION_SIZE}")
    print(f"  Num Islands:     {NUM_ISLANDS}")
    print(f"  Log Level:       {LOG_LEVEL}")
    print(f"  LLM Model:       {config['llm']['models'][0]['name']}")
    print(f"  LLM API:         {config['llm']['api_base'].split('/')[-1]}")
    print(f"  Max Tokens:      {config['llm'].get('max_tokens', 'NOT SET')}")
    print(f"  Timeout:         {config['llm'].get('timeout', 'NOT SET')}s")
    print(f"  LLM Sampling:    {config['llm']['sampling']['fn']}")
    print(f"  Prompt Dir:      {config['prompt']['template_dir']}")
    print(f"  Database Path:   {config['database']['db_path']}")
    print(f"  LLM  Feedback:   {config['evaluator']['use_llm_feedback']}")
    print(f"  Parallel  Evaluation:   {config['evaluator']['parallel_evaluations']}")
    
    # Debug: Check prompt configuration
    print(f"\n🔍 Prompt Configuration Details:")
    if 'system_message' in config['prompt']:
        print(f"  ⚠️  Inline system_message present (will override template!)")
    else:
        print(f"  ✅ No inline system_message (will load from template)")
    
    if 'evaluator_system_message' in config['prompt']:
        print(f"  ⚠️  Inline evaluator_system_message present (will override template!)")
    else:
        print(f"  ✅ No inline evaluator_system_message (will load from template)")
    
    # Verify template files exist
    template_dir_path = TUTORIAL_DIR / config['prompt']['template_dir'].lstrip('./')
    sys_msg_file = template_dir_path / "system_message.txt"
    eval_msg_file = template_dir_path / "evaluator_system_message.txt"
    
    print(f"\n📁 Template Files:")
    print(f"  {sys_msg_file.name}: {'✅ EXISTS' if sys_msg_file.exists() else '❌ MISSING'}")
    print(f"  {eval_msg_file.name}: {'✅ EXISTS' if eval_msg_file.exists() else '❌ MISSING'}")
    
    if sys_msg_file.exists():
        with open(sys_msg_file, 'r') as f:
            content = f.read()
            print(f"\n  system_message.txt: {len(content)} chars")
            if "ALGORITHMIC IMPROVEMENTS" in content or "OPERATOR FUSION" in content:
                print(f"  ✅ Contains advanced prompt keywords!")
            else:
                print(f"  ❌ Does not contain expected keywords")
    
    print(f"\n✅ Ready to run evolution!")
else:
    print("❌ No config template found!")
    CONFIG_FILE = None


✅ Found config template: configs/default_config.yaml
✅ Configuration saved to: tutorial_config.yaml

📝 Evolution Parameters:
  Max Iterations:  10
  Population Size: 50
  Num Islands:     4
  Log Level:       WARNING
  LLM Model:       claude-sonnet-4-5
  LLM API:         claude-sonnet-4-5
  Max Tokens:      10000
  Timeout:         200s
  LLM Sampling:    random
  Prompt Dir:      ./prompts_tutorial
  Database Path:   program_database
  LLM  Feedback:   True
  Parallel  Evaluation:   1

🔍 Prompt Configuration Details:
  ✅ No inline system_message (will load from template)
  ✅ No inline evaluator_system_message (will load from template)

📁 Template Files:
  system_message.txt: ✅ EXISTS
  evaluator_system_message.txt: ✅ EXISTS

  system_message.txt: 14058 chars
  ✅ Contains advanced prompt keywords!

✅ Ready to run evolution!


### 📄 Step 6.5: Preview Prompts (Optional - for debugging)

Run this cell to see what system messages will be sent to the LLM.


In [9]:
# OPTIONAL: Preview what prompts will be sent to the LLM
import sys
from pathlib import Path

# Add OpenEvolve to path
sys.path.insert(0, str(Path.cwd().parent))

try:
    from openevolve.prompt.templates import TemplateManager
    import yaml
    
    TUTORIAL_DIR = Path.cwd()
    
    with open('tutorial_config.yaml', 'r') as f:
        config = yaml.safe_load(f)
    
    template_dir = config['prompt'].get('template_dir')
    
    if template_dir:
        print("="*80)
        print("🔍 PROMPTS THAT WILL BE SENT TO LLM")
        print("="*80)
        
        # Resolve relative path
        if template_dir.startswith('./'):
            template_path = TUTORIAL_DIR / template_dir.lstrip('./')
        else:
            template_path = Path(template_dir)
        
        print(f"\nTemplate directory: {template_path}")
        print(f"Directory exists: {template_path.exists()}")
        
        if template_path.exists():
            sys_msg_file = template_path / "system_message.txt"
            eval_msg_file = template_path / "evaluator_system_message.txt"
            
            print(f"\n" + "-"*80)
            print(f"📝 SYSTEM MESSAGE (for code generation)")
            print(f"-"*80)
            
            if sys_msg_file.exists():
                with open(sys_msg_file, 'r') as f:
                    sys_msg = f.read()
                
                print(f"✅ File: {sys_msg_file.name}")
                print(f"✅ Length: {len(sys_msg)} characters")
                print(f"\n--- First 800 characters ---")
                print(sys_msg[:800])
                print("\n... [full prompt will be sent to LLM]")
                
                # Check for key content
                if "ALGORITHMIC" in sys_msg:
                    print("\n✅ Contains: ALGORITHMIC optimization guidance")
                if "OPERATOR FUSION" in sys_msg:
                    print("✅ Contains: OPERATOR FUSION technique")
                if "tl.float32" in sys_msg:
                    print("✅ Contains: tl.float32 syntax rules (prevents errors!)")
            else:
                print(f"❌ File not found: {sys_msg_file}")
            
            print(f"\n" + "-"*80)
            print(f"📝 EVALUATOR SYSTEM MESSAGE (for feedback)")
            print(f"-"*80)
            
            if eval_msg_file.exists():
                with open(eval_msg_file, 'r') as f:
                    eval_msg = f.read()
                
                print(f"✅ File: {eval_msg_file.name}")
                print(f"✅ Length: {len(eval_msg)} characters")
                print(f"\n--- First 500 characters ---")
                print(eval_msg[:500])
                print("\n... [full prompt will be sent to LLM]")
            else:
                print(f"❌ File not found: {eval_msg_file}")
    else:
        print("⚠️  No template_dir configured - using inline prompts or defaults")

except Exception as e:
    print(f"❌ Error: {e}")
    print(f"\n⚠️  This is an optional debug cell - you can skip it if needed")


🔍 PROMPTS THAT WILL BE SENT TO LLM

Template directory: /home/sapmajum/neurips/geak-openevolve/tutorial/prompts_tutorial
Directory exists: True

--------------------------------------------------------------------------------
📝 SYSTEM MESSAGE (for code generation)
--------------------------------------------------------------------------------
✅ File: system_message.txt
✅ Length: 14058 characters

--- First 800 characters ---
Role: GPU Kernel Optimization Expert - Focus on Algorithmic Improvements

You are optimizing Triton GPU kernels for AMD ROCm. Your goal is to achieve 2-5x speedup through smart algorithmic changes.

═══════════════════════════════════════════════════════════════════════════════
CRITICAL TRITON SYNTAX RULES (Follow These to Avoid Errors!)
═══════════════════════════════════════════════════════════════════════════════

📘 DTYPES (triton.language types)
─────────────────────────────────
Available: tl.float16, tl.float32, tl.float64, tl.bfloat16
           tl.int8, tl.

In [10]:
# Step 7: Setup Output Directory and Validate
from pathlib import Path
from datetime import datetime

OPENEVOLVE_ROOT = Path.cwd().parent
TUTORIAL_DIR = OPENEVOLVE_ROOT / "tutorial"

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
OUTPUT_DIR = TUTORIAL_DIR / "runs" / f"tutorial_run_{timestamp}"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# CRITICAL: Ensure evals directory exists (needed by evaluator)
EVALS_DIR = TUTORIAL_DIR / "evals"
EVALS_DIR.mkdir(exist_ok=True)

print(f"✅ Output directory: {OUTPUT_DIR.relative_to(TUTORIAL_DIR)}")
print(f"✅ Evals directory: {EVALS_DIR.relative_to(TUTORIAL_DIR)}")

print("\n" + "="*70)
print("📋 Pre-Flight Check")
print("="*70)

try:
    kernel_var = INITIAL_KERNEL
    kernel_defined = True
except NameError:
    kernel_var = None
    kernel_defined = False

try:
    evaluator_var = EVALUATOR_PATH
    evaluator_defined = True
except NameError:
    evaluator_var = None
    evaluator_defined = False

try:
    config_var = CONFIG_FILE
    config_defined = True
except NameError:
    config_var = None
    config_defined = False

components = {
    "Kernel": (kernel_var, kernel_defined),
    "Evaluator": (evaluator_var, evaluator_defined),
    "Config": (config_var, config_defined)
}

all_ready = True
missing_cells = []

for name, (path, is_defined) in components.items():
    if not is_defined:
        print(f"❌ {name:12s}: NOT DEFINED (run earlier cell)")
        all_ready = False
        if name == "Kernel":
            missing_cells.append("Cell 5")
        elif name == "Evaluator":
            missing_cells.append("Cell 6")
        elif name == "Config":
            missing_cells.append("Cell 7")
    elif path and Path(path).exists():
        print(f"✅ {name:12s}: {Path(path).name}")
    else:
        print(f"❌ {name:12s}: NOT FOUND")
        all_ready = False

print("="*70)

if all_ready:
    print("\n🚀 All components ready! You can proceed to run evolution.")
else:
    print("\n⚠️  Some components are missing!")
    if missing_cells:
        print("\n📝 Please run these cells first:")
        for cell in missing_cells:
            print(f"   • {cell}")


✅ Output directory: runs/tutorial_run_20251126_210144
✅ Evals directory: evals

📋 Pre-Flight Check
✅ Kernel      : test_add_kernel.py
✅ Evaluator   : rocm_evaluator.py
✅ Config      : tutorial_config.yaml

🚀 All components ready! You can proceed to run evolution.


In [11]:
# Step 8: Run OpenEvolve Evolution
import subprocess
import os
from pathlib import Path

if not (INITIAL_KERNEL and EVALUATOR_PATH and CONFIG_FILE):
    print("❌ Missing required components!")
    print(f"   Kernel:    {INITIAL_KERNEL is not None and Path(INITIAL_KERNEL).exists()}")
    print(f"   Evaluator: {EVALUATOR_PATH is not None and Path(EVALUATOR_PATH).exists()}")
    print(f"   Config:    {CONFIG_FILE is not None and Path(CONFIG_FILE).exists()}")
else:
    command = [
        "openevolve-run",
        str(INITIAL_KERNEL),
        str(EVALUATOR_PATH),
        "--config", str(CONFIG_FILE),
        "--output", str(OUTPUT_DIR)
    ]
    
    print("🚀 Starting OpenEvolve Evolution...")
    print("="*70)
    print(f"📦 Kernel:    {Path(INITIAL_KERNEL).name}")
    print(f"⚙️  Evaluator: {Path(EVALUATOR_PATH).name}")
    print(f"📋 Config:    {Path(CONFIG_FILE).name}")
    print(f"📁 Output:    {OUTPUT_DIR.relative_to(TUTORIAL_DIR)}")
    print(f"🏠 Working Dir: {TUTORIAL_DIR}")
    print("="*70)
    print(f"\n$ cd {TUTORIAL_DIR}")
    print(f"$ {' '.join(command)}\n")
    print("="*70)
    
    # CRITICAL: Run from tutorial directory where evals/ exists
    result = subprocess.run(
        command, 
        capture_output=False, 
        text=True,
        cwd=str(TUTORIAL_DIR)  # Run from tutorial directory
    )
    
    print("="*70)
    if result.returncode == 0:
        print("\n✅ Evolution completed successfully!")
        print(f"\n📊 Results saved to: {OUTPUT_DIR.relative_to(TUTORIAL_DIR)}")
    else:
        print(f"\n❌ Evolution failed with exit code: {result.returncode}")


🚀 Starting OpenEvolve Evolution...
📦 Kernel:    test_add_kernel.py
⚙️  Evaluator: rocm_evaluator.py
📋 Config:    tutorial_config.yaml
📁 Output:    runs/tutorial_run_20251126_210144
🏠 Working Dir: /home/sapmajum/neurips/geak-openevolve/tutorial

$ cd /home/sapmajum/neurips/geak-openevolve/tutorial
$ openevolve-run /home/sapmajum/neurips/geak-openevolve/GEAK-eval-OE/geak_eval/data/ROCm/data/ROCm_v1/test_add_kernel.py /home/sapmajum/neurips/geak-openevolve/examples/tb/rocm_evaluator.py --config /home/sapmajum/neurips/geak-openevolve/tutorial/tutorial_config.yaml --output /home/sapmajum/neurips/geak-openevolve/tutorial/runs/tutorial_run_20251126_210144

✅ Loaded template 'evaluator_system_message' from prompts_tutorial/evaluator_system_message.txt (7482 chars)
✅ Loaded template 'system_message' from prompts_tutorial/system_message.txt (14058 chars)
✅ Loaded template 'evaluator_system_message' from prompts_tutorial/evaluator_system_message.txt (7482 chars)
✅ Loaded template 'system_message'

2025-11-26 21:01:47,740 - INFO - Adding initial program to database
2025-11-26 21:01:54,123 - INFO - Time spent in evaluation: 6.38 seconds
2025-11-26 21:01:54,123 - INFO - 🎯 Using system_message from template override: evaluator_system_message (7482 chars)
2025-11-26 21:02:02,293 - INFO - HTTP Request: POST https://llm-api.amd.com/AnthropicVertex/deployments/claude-sonnet-4-5/chat/completions "HTTP/1.1 200 OK"
2025-11-26 21:02:02,302 - INFO - Time spent in LLM evaluation: 8.18 seconds
2025-11-26 21:02:02,302 - INFO - Evaluated program 769f8938-705d-460e-98c2-f43cb14cfff8 in 8.18s: success=1.0000, final_score=1.0000, performance_metrics=1.0000, correctness_score=1.0000, combined_score=1.0000, benchmark_results=['Performance report: Kernel parameters: SIZE=98432; BLOCK_SIZE_RUNTIME=1024; dtype_str=float16, achieved latency: 0.006900 ms, speedup: 1.0000x.'], baseline_comparison=Performance report: Kernel parameters: SIZE=98432; BLOCK_SIZE_RUNTIME=1024; dtype_str=float16, achieved latency

⚠️ WARNING: Kernel evaluator path /home/sapmajum/neurips/geak-openevolve/GEAK-eval-OE/geak_eval/data/ROCm/data/ROCm_v1/evaluator.py does not exist, using default given path.
🛡️ BULLETPROOF TRITON KERNEL EVALUATOR (AMD GPU) INITIALISED
Using program text from file: /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmpsyhi6i1s/test_add_kernel.py
Evaluating Triton kernel from: /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmpsyhi6i1s/test_add_kernel.py
📝 Extracted kernel name from program_text: test_add_kernel.py
📝 Final kernel name for test merging: test_add_kernel.py
✅ No @triton.autotune - using ROCm_v1 tests
✅ Merged kernel with test code from ROCm_v1
Running correctness tests: pytest -v -x --maxfail=1 /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmpsyhi6i1s/test_add_kernel.py -k not (test_performance or test_save)
Running performance tests: pytest -v -x --maxfail=1 /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmpsyhi6i1s/test_add_kernel.py -k test_perfor

2025-11-26 21:04:21,863 - INFO - Time spent in evaluation: 10.89 seconds
2025-11-26 21:04:21,863 - INFO - 🎯 Using system_message from template override: evaluator_system_message (7482 chars)
2025-11-26 21:04:31,721 - INFO - HTTP Request: POST https://llm-api.amd.com/AnthropicVertex/deployments/claude-sonnet-4-5/chat/completions "HTTP/1.1 200 OK"
2025-11-26 21:04:31,723 - INFO - Time spent in LLM evaluation: 9.86 seconds
2025-11-26 21:04:31,723 - INFO - Evaluated program 803e00b0-ff9b-4ad5-ab92-70199f5c6e61 in 9.86s: success=1.0000, final_score=1.0455, performance_metrics=1.0455, correctness_score=1.0000, combined_score=1.0455, benchmark_results=['Performance report: Kernel parameters: SIZE=98432; BLOCK_SIZE_RUNTIME=1024; dtype_str=float16, achieved latency: 0.006600 ms, speedup: 1.0455x.'], baseline_comparison=Performance report: Kernel parameters: SIZE=98432; BLOCK_SIZE_RUNTIME=1024; dtype_str=float16, achieved latency: 0.006600 ms, speedup: 1.0455x. Speedup=1.0455x (baseline: 0.00690


Using program text from file: /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmp66h_ijs8/test_add_kernel.py
Evaluating Triton kernel from: /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmp66h_ijs8/test_add_kernel.py
📝 Extracted kernel name from program_text: test_add_kernel.py
📝 Final kernel name for test merging: test_add_kernel.py
✅ Detected @triton.autotune - using ROCm_v1_autotune tests
✅ Merged kernel with test code from ROCm_v1_autotune
Running correctness tests: pytest -v -x --maxfail=1 /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmp66h_ijs8/test_add_kernel.py -k not (test_performance or test_save)
Running performance tests: pytest -v -x --maxfail=1 /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmp66h_ijs8/test_add_kernel.py -k test_performance or test_save_performance_results
Performance test result - returncode: 0
Performance test stdout: ============================= test session starts ==============================
platform linux -- Python 

...


Running performance tests: pytest -v -x --maxfail=1 /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmpspicarim/test_add_kernel.py -k test_performance or test_save_performance_results
Performance test result - returncode: 0
Performance test stdout: ============================= test session starts ==============================
platform linux -- Python 3.13.9, pytest-9.0.1, pluggy-1.5.0 -- /home/sapmajum/miniconda3/bin/python3.13
cachedir: .pytest_cache
rootdir: /home/sapmajum/neurips/geak-openevolve
configfile: pyproject.toml
plugins: timeout-2.4.0, anyio-4.11.0
collecting ... collected 8 items / 3 deselected / 5 selected

evals/tmpspicarim/test_add_kernel.py::test_performance[98432-1024-float16] PASSED [ 20%
Performance test stderr: 
Looking for performance results in: /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmpspicarim/perf
Found 1 JSON files: ['add_kernel_perf.json']
Reading performance data from: /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmpspicari

2025-11-26 21:38:18,602 - INFO - Time spent in evaluation: 17.67 seconds
2025-11-26 21:38:18,602 - INFO - 🎯 Using system_message from template override: evaluator_system_message (7482 chars)
2025-11-26 21:38:29,361 - INFO - HTTP Request: POST https://llm-api.amd.com/AnthropicVertex/deployments/claude-sonnet-4-5/chat/completions "HTTP/1.1 200 OK"
2025-11-26 21:38:29,363 - INFO - Time spent in LLM evaluation: 10.76 seconds
2025-11-26 21:38:29,363 - INFO - Evaluated program 5ee7099e-6504-48d3-953c-438630ee1613 in 10.76s: success=1.0000, final_score=1.0299, performance_metrics=1.0299, correctness_score=1.0000, combined_score=1.0299, benchmark_results=['Performance report: Kernel parameters: SIZE=98432; BLOCK_SIZE_RUNTIME=1024; dtype_str=float16, achieved latency: 0.006700 ms, speedup: 1.0299x.'], baseline_comparison=Performance report: Kernel parameters: SIZE=98432; BLOCK_SIZE_RUNTIME=1024; dtype_str=float16, achieved latency: 0.006700 ms, speedup: 1.0299x. Speedup=1.0299x (baseline: 0.006


Performance test result - returncode: 0
Performance test stdout: ============================= test session starts ==============================
platform linux -- Python 3.13.9, pytest-9.0.1, pluggy-1.5.0 -- /home/sapmajum/miniconda3/bin/python3.13
cachedir: .pytest_cache
rootdir: /home/sapmajum/neurips/geak-openevolve
configfile: pyproject.toml
plugins: timeout-2.4.0, anyio-4.11.0
collecting ... collected 8 items / 3 deselected / 5 selected

evals/tmpzg9tveuc/test_add_kernel.py::test_performance[98432-1024-float16] PASSED [ 20%
Performance test stderr: 
Looking for performance results in: /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmpzg9tveuc/perf
Found 1 JSON files: ['add_kernel_perf.json']
Reading performance data from: /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmpzg9tveuc/perf/add_kernel_perf.json (most recent)
Performance data structure: ['params', 'ms', 'min_ms', 'max_ms', 'GB/s', 'TFLOPS']
Performance: 0.0067ms (from key 'ms')
Loaded baseline latency fro

2025-11-26 21:39:56,889 - INFO - Time spent in evaluation: 13.22 seconds
2025-11-26 21:39:56,889 - INFO - 🎯 Using system_message from template override: evaluator_system_message (7482 chars)
2025-11-26 21:40:06,022 - INFO - HTTP Request: POST https://llm-api.amd.com/AnthropicVertex/deployments/claude-sonnet-4-5/chat/completions "HTTP/1.1 200 OK"
2025-11-26 21:40:06,024 - INFO - Time spent in LLM evaluation: 9.13 seconds
2025-11-26 21:40:06,024 - INFO - Evaluated program 9d056eca-d958-4279-861f-f03d23efe943 in 9.13s: success=1.0000, final_score=1.0299, performance_metrics=1.0299, correctness_score=1.0000, combined_score=1.0299, benchmark_results=['Performance report: Kernel parameters: SIZE=98432; BLOCK_SIZE_RUNTIME=1024; dtype_str=float16, achieved latency: 0.006700 ms, speedup: 1.0299x.'], baseline_comparison=Performance report: Kernel parameters: SIZE=98432; BLOCK_SIZE_RUNTIME=1024; dtype_str=float16, achieved latency: 0.006700 ms, speedup: 1.0299x. Speedup=1.0299x (baseline: 0.00690


Performance test stderr: 
Looking for performance results in: /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmp9vhy9dpu/perf
Found 1 JSON files: ['add_kernel_perf.json']
Reading performance data from: /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmp9vhy9dpu/perf/add_kernel_perf.json (most recent)
Performance data structure: ['params', 'ms', 'min_ms', 'max_ms', 'GB/s', 'TFLOPS']
Performance: 0.0067ms (from key 'ms')
Loaded baseline latency from file: 0.006900ms
Calculated speedup: 0.006900ms / 0.006700ms = 1.0299x
🛡️ BULLETPROOF TRITON KERNEL EVALUATOR (AMD GPU) INITIALISED
Using program text from file: /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmpcnbq2l0w/test_add_kernel.py
Evaluating Triton kernel from: /home/sapmajum/neurips/geak-openevolve/tutorial/evals/tmpcnbq2l0w/test_add_kernel.py
📝 Extracted kernel name from program_text: test_add_kernel.py
📝 Final kernel name for test merging: test_add_kernel.py
✅ Detected @triton.autotune - using ROCm_v1_autotune